In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display

In [2]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .widget-label { 
        font-size: 24px !important;
    }
    .slider-value { 
        font-size: 24px !important;
    }
    .widget-hslider .widget-readout {
        font-size: 22px !important;
    }
</style>
"""))

In [ ]:
grid_size = 2000
circle_radius = 400

def simulate_N_receivers(angle_deg, frequency, N):
    rN_x = np.zeros(N)
    if N==1:
        rN_y = np.array([0])
    else:
        rN_y = np.linspace(-circle_radius/10, circle_radius/10, N)

    sampling_rate = 16000
    t = np.linspace(0, 6, sampling_rate*6)
    signal = np.sin(t*2*np.pi*frequency)
    start = 3
    duration = 2*(10**(-np.floor(np.log10(frequency))))
    end = start+duration
    speed_of_sound_in_water = 1500

    angle_rad = np.radians(angle_deg)
    circle_x = circle_radius * np.cos(angle_rad)
    circle_y = circle_radius * np.sin(angle_rad)

    dist_from_source_to_rN = np.sqrt(((circle_x-rN_x)**2) + ((circle_y-rN_y)**2))
    time_to_rN = dist_from_source_to_rN / speed_of_sound_in_water
    receive_signalN = signal[:,np.newaxis]/(dist_from_source_to_rN**2)
    rN_inds_begin = np.floor((start-time_to_rN)*sampling_rate).astype(int)
    rN_inds_end = rN_inds_begin + np.ceil(duration*sampling_rate).astype(int)
    trace_offsets = 2e-4 * rN_y / circle_radius
    fig = plt.figure(figsize=(15, 5))
    gs = gridspec.GridSpec(1, 3, width_ratios=[4, 4, 2], wspace=0.3) 
    gs_signals = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.8)
    map_ax = plt.subplot(gs[0, 0])
    map_ax.scatter(circle_x, circle_y, edgecolors='red', facecolors='none', linewidths=2, s=50, label=f"Source")

    received_ax = plt.subplot(gs_signals[0])
    summed_signal = np.zeros(np.ceil(duration*sampling_rate).astype(int))
    for i, rn_ind in enumerate(rN_inds_begin):
        chunk_of_received_signal_n_in_window = receive_signalN[rn_ind:np.ceil(rn_ind+(duration*sampling_rate)).astype(int), i]
        summed_signal+=chunk_of_received_signal_n_in_window
        map_ax.scatter(rN_x[i], rN_y[i], s=10, label=f"Receiver #{int(i+1)}")
        received_ax.plot((1000*(t+time_to_rN[i]-start)), receive_signalN[:,i]+trace_offsets[i], label=f'rcvr #{int(i+1)}')

    vrms_summed_fixed = np.sqrt(np.mean(np.square(summed_signal)))
    vrms_summed_fixed_dB = 20*np.log10(vrms_summed_fixed)

    received_ax.set_xlabel("Time (ms)")
    received_ax.set_ylabel("Amplitude")
    received_ax.set_title("Received voltage signal")
    received_ax.legend(ncol=3, loc='upper right')
    received_ax.grid(which='both')
    received_ax.set_xlim(0, 1000*duration)
    received_ax.set_ylim(-5e-5, 5e-5)

    summed_ax = plt.subplot(gs_signals[1])
    summed_ax.plot(1000*(t[rN_inds_begin[N-1]:rN_inds_end[N-1]]+time_to_rN[N-1]-start), summed_signal, 'k')
    summed_ax.set_xlabel("Time (ms)")
    summed_ax.set_ylabel("Amplitude")
    summed_ax.set_title("Summed voltage signal")
    summed_ax.grid(which='both')
    summed_ax.set_xlim(0, 1000*duration)
    summed_ax.set_ylim(-5e-5, 5e-5)

    map_ax.set_xlim(-grid_size / 2, grid_size / 2)
    map_ax.set_ylim(-grid_size / 2, grid_size / 2)
    map_ax.set_xlabel("X (meters)")
    map_ax.set_ylabel("Y (meters)")
    map_ax.set_title(f"Sound Reception ({int(N)} receivers; 1 source)")
    map_ax.legend(ncol=2, loc='upper right')

    angle_degrees = np.arange(-90, 90)
    angle_radians = np.radians(angle_degrees)
    circle_x_arr = circle_radius * np.cos(angle_radians)
    circle_y_arr = circle_radius * np.sin(angle_radians)

    dist_from_source_to_r_n_arr = np.sqrt((circle_x_arr[None, :] - rN_x[:, None]) ** 2 + (circle_y_arr[None, :] - rN_y[:, None]) ** 2)
    dist_from_source_to_r_n_arr = np.maximum(dist_from_source_to_r_n_arr, 1.0)

    time_to_rn_array = dist_from_source_to_r_n_arr / speed_of_sound_in_water
    receive_signaln_for_degrees = signal[:, None, None] / (dist_from_source_to_r_n_arr[None, :, :] ** 2)
    rn_indices = ((start - time_to_rn_array) * sampling_rate).astype(int) + np.arange(int(duration * sampling_rate))[:, None, None]
    chunk_of_received_signaln_in_window_degrees = np.take_along_axis(receive_signaln_for_degrees, rn_indices, axis=0)
    summed_signal_degrees = np.sum(chunk_of_received_signaln_in_window_degrees, axis=1)

    vrms_summed = np.sqrt(np.mean(np.square(summed_signal_degrees), axis=0))
    vrms_summed_dB = 20*np.log10(vrms_summed)

    gs_beam_pattern = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs[0, 2], hspace=0.8)
    beam_pattern_ax = plt.subplot(gs_beam_pattern[0])
    beam_pattern_ax.set_title(f'Beam pattern')
    beam_pattern_ax.plot(vrms_summed_dB, angle_degrees, color='k')
    beam_pattern_ax.scatter(vrms_summed_fixed_dB, angle_deg, edgecolors='red', facecolors='w', linewidth=2, zorder=2)
    beam_pattern_ax.set_xlim(-120, -90)
    beam_pattern_ax.grid(which='both')
    beam_pattern_ax.set_xlabel(r'dB$V_{rms}$ (dB)')
    beam_pattern_ax.set_ylabel('Angle (°)')
    
    plt.show()


angle_slider = widgets.IntSlider(
    value=0, min=-90, max=90, step=1, description="Angle (°)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

frequency_slider = widgets.IntSlider(
    value=0, min=1, max=1e3, step=1, description="Frequency (Hz)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

N_slider = widgets.IntSlider(
    value=0, min=1, max=10, step=1, description="# receivers", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

interactive_plot = widgets.interactive(simulate_N_receivers, angle_deg=angle_slider, frequency=frequency_slider, N=N_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='1000px'), max=90, min=-…